In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Verileri yükleme
ages = pd.read_csv('Ages.csv')
data = pd.read_csv('data.csv')

# 'Sample Accession.1' sütununu çıkarma
data = data.drop(columns=['Sample Accession.1'])

# Sütun adlarını yeniden adlandırma
ages.columns = ['Sample_Accession', 'Age']
data.columns = ['Sample_Accession'] + list(data.columns[1:])

# Yaş verilerini veri setine birleştirme
data = data.merge(ages, on='Sample_Accession')

# İlk birkaç satırı gözlemleyelim
print(data.head())

# Eksik değerlerin kontrolü
print(data.isnull().sum())

# Eksik değerlerin çıkarılması (eğer varsa)
data = data.dropna()

# Yaş dağılımını görselleştirme
plt.figure(figsize=(10, 6))
sns.histplot(ages['Age'], bins=30, kde=True)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

# Model için veri hazırlığı

# Sadece sayısal özelliklerin seçilmesi
X = data.drop(columns=['Sample_Accession', 'Age'])
y = data['Age']

# Veriyi eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Random Forest

# Random Forest modeli oluşturma ve eğitme
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Tahmin yapma
rf_predictions = rf_model.predict(X_test)

# Performans Metrikleri
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print(f'Random Forest MAE: {rf_mae}, R2: {rf_r2}')

# Sonuçların görselleştirilmesi
plt.figure(figsize=(12, 6))
plt.scatter(y_test, rf_predictions, alpha=0.5, label='Random Forest')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')
plt.xlabel('Gerçek Yaş')
plt.ylabel('Tahmin Edilen Yaş')
plt.legend()
plt.title('Gerçek Yaş vs Tahmin Edilen Yaş (Random Forest)')
plt.show()


# Hiperparametre ızgarası
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_features': [7, 19, 21],
    'min_samples_split':[3,7,11],
    'max_depth':[3,7,11]
}

# Grid Search ile en iyi parametreleri bulma
rf_grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid_rf, cv=3, n_jobs=-1, verbose=2)
rf_grid_search.fit(X_train, y_train)

# En iyi parametreler ile modeli yeniden eğitme
best_rf_model = rf_grid_search.best_estimator_
best_rf_predictions = best_rf_model.predict(X_test)

# Performans Metrikleri
best_rf_mae = mean_absolute_error(y_test, best_rf_predictions)
best_rf_r2 = r2_score(y_test, best_rf_predictions)


print(f'Best Random Forest MAE: {best_rf_mae}, R2: {best_rf_r2}')

# Gradient Boost

# Hiperparametre ızgarası
param_grid_gb = {
    'learning_rate': [0.1, 0.01, 0.05],
            'n_estimators': [100,200],
            'max_depth':[2,3,5,8]
}

# Grid Search ile en iyi parametreleri bulma
gb_grid_search = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42), param_grid=param_grid_gb, cv=3, n_jobs=-1, verbose=2)
gb_grid_search.fit(X_train, y_train)

# En iyi parametreler ile modeli yeniden eğitme
best_gb_model = gb_grid_search.best_estimator_
best_gb_predictions = best_gb_model.predict(X_test)

# Performans Metrikleri
best_gb_mae = mean_absolute_error(y_test, best_gb_predictions)
best_gb_r2 = r2_score(y_test, best_gb_predictions)

print(f'Best Gradient Boosting MAE: {best_gb_mae}, R2: {best_gb_r2}')

# XGB

param_grid_xgb = {'learning_rate': [0.1, 0.01, 0.001],
            'subsample':[0.6, 0.8, 1],
            'n_estimators': [100,500],
            'max_depth':[3,5,7]
}

xgb_grid_search = GridSearchCV(estimator=xgb.XGBRegressor(random_state=42),
                               param_grid=param_grid_xgb,
                               cv=10,
                               n_jobs=-1,
                               verbose=2)
xgb_grid_search.fit(X_train, y_train)

# En iyi parametreler ile modeli yeniden eğitme
best_xgb_model = xgb_grid_search.best_estimator_

best_xgb_predictions = best_xgb_model.predict(X_test)

# Performans Metrikleri
best_xgb_mae = mean_absolute_error(y_test, best_xgb_predictions)
best_xgb_r2 = r2_score(y_test, best_xgb_predictions)

print(f'Best XGBoost MAE: {best_xgb_mae}, R2: {best_xgb_r2}')

# Sonuç Görselleştirilmesi

# Random Forest için sonuçların görselleştirilmesi
plt.figure(figsize=(12, 6))
plt.scatter(y_test, best_rf_predictions, alpha=0.5, label='Optimized Random Forest')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')
plt.xlabel('Gerçek Yaş')
plt.ylabel('Tahmin Edilen Yaş')
plt.legend()
plt.title('Gerçek Yaş vs Tahmin Edilen Yaş (Optimized Random Forest)')
plt.show()

# Gradient Boosting için sonuçların görselleştirilmesi
plt.figure(figsize=(12, 6))
plt.scatter(y_test, best_gb_predictions, alpha=0.5, label='Optimized Gradient Boosting')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')
plt.xlabel('Gerçek Yaş')
plt.ylabel('Tahmin Edilen Yaş')
plt.legend()
plt.title('Gerçek Yaş vs Tahmin Edilen Yaş (Optimized Gradient Boosting)')
plt.show()

# Sonuçların görselleştirilmesi
plt.figure(figsize=(12, 6))
plt.scatter(y_test, xgb_predictions, alpha=0.5, label='XGBoost')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--', color='red')
plt.xlabel('Gerçek Yaş')
plt.ylabel('Tahmin Edilen Yaş')
plt.legend()
plt.title('Gerçek Yaş vs Tahmin Edilen Yaş (XGBoost)')
plt.show()

